# Ikeda for many ships
There are some interesting section data in the manoeuvring database, here it is investigated if this can be used as input to *ScoresII*.

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score



In [ ]:
df_sections = pd.read_csv('sections.csv', sep=';', index_col=0)
mask=df_sections['b20'].notnull()
df_sections=df_sections.loc[mask].copy()
df_sections.drop(index=['DAE18A','DAE18B'], inplace=True)  # Bad project number
df_sections['Project No']=df_sections['Project No'].astype(int)

In [ ]:
df_data = pd.read_csv('ship_data.csv', sep=';', index_col=0)

In [ ]:
df_data.head()

In [ ]:
df_sections_data=pd.merge(left=df_sections, right=df_data, how='left', left_on='SHIP', right_on='SHIP', 
         suffixes=('','_data'))
df_sections_data.set_index('SHIP', inplace=True)
assert df_sections_data.index.is_unique


In [ ]:
df_sections_data.describe()

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
interesting_projects = set(df_rolldecay['project_number']) & set(df_sections_data['Project No'])
interesting_projects

In [ ]:
df_=df_sections_data.reset_index()
df = pd.merge(left=df_, right=df_rolldecay, how='inner', left_on=('Project No','DISP'), 
         right_on=('project_number','Volume'), suffixes=('','_model'))

In [ ]:
df.describe()

In [ ]:
mask=df['id'].notnull()
df = df.loc[mask].copy()

In [ ]:
def assemble_sections(row, lpp):
    
    b=np.zeros(21)
    area=np.zeros(21)
    t=np.zeros(21)
    for i in range(21):
        b_key='b%i' % i
        area_key='area%i' % i
        t_key = 't%i' % i
        b[i]=row[b_key]
        area[i]=row[area_key]
        t[i]=row[t_key]
    
    df_sections = pd.DataFrame()
    df_sections['b']=b
    df_sections['area']=area
    df_sections['t']=t
    df_sections['x']=np.linspace(0,lpp,21)
    df_sections['SHIP']=int(row['SHIP'])
    df_sections['Project No']=row['Project No']
    
    df_sections.dropna(inplace=True)
    return df_sections

In [ ]:
df_sections_data

In [ ]:
df_all_sections = pd.DataFrame()

for index, row in df_sections.iterrows():
    data = df_sections_data.loc[row.SHIP]
    lpp=data.LPP
    df_ = assemble_sections(row=row, lpp=lpp)
    df_all_sections=df_all_sections.append(df_)


In [ ]:
df_all_sections.head()

In [ ]:
def count_variations(df_group):
    df_group=df_group.copy()
     
    for i in range(21):
        b_key='b%i' % i
        area_key='area%i' % i
        t_key = 't%i' % i
    
        keys=[b_key, area_key, t_key]
        
        for key in keys:
            variations = len(df_group[key].unique())
            if not variations==1:
                break
        
        if not variations==1:
                break
            
    return variations
    

In [ ]:
df.groupby(by='Project No').apply(func=count_variations)            

In [ ]:
mask = ~df.duplicated(['SHIP','loading_condition_id'], keep='first')
df_ids = df.loc[mask,['SHIP','loading_condition_id']].copy()
df_ids.set_index('SHIP', inplace=True)
assert df_ids.index.is_unique

In [ ]:
loading_condition_ids = df_ids['loading_condition_id']

In [ ]:
df_all_sections=pd.merge(left=df_all_sections, right=loading_condition_ids, how='outer', left_on='SHIP', right_index=True)

## Run ScoresII for one of the loading conditions

In [ ]:
row=df.iloc[0]
sections = df_all_sections.groupby('SHIP').get_group(row.SHIP)

fig,axes=plt.subplots(nrows=3)
ax=axes[0]
ax.set_title('Project: %s loading_condition: %i' % (row.project_number, row.loading_condition_id))
sections.plot(x='x', y='b', ax=ax);
ax.plot([0,row.LPP], [row.B,row.B], 'r--')

ax=axes[1]
sections.plot(x='x', y='t', ax=ax);
ax.plot([0,row.LPP], [row.TF,row.TF], 'r--')

ax=axes[2]
sections.plot(x='x', y='area', ax=ax);
plt.tight_layout()

In [ ]:
row

In [ ]:
from pyscores2.indata import Indata
from pyscores2.runScores2 import Calculation
from pyscores2.output import OutputFile
from pyscores2 import TDPError
from rolldecayestimators.ikeda import Ikeda, IkedaR

In [ ]:
sections['cScores']=sections['area']/(sections['b']*sections['t'])
mask=sections['cScores']>1
sections[mask]=1
sections

In [ ]:
indata = Indata()

indata.cScores=sections['cScores']
indata.ts=sections['t']
indata.bs=sections['b']
indata.zbars=np.zeros_like(sections['b'])  # Guessing...

beam=sections['b'].max()
indata.lpp=row.lpp
indata.displacement=row.Volume
draught=(row.TA+row.TF)/2
indata.draught=draught
g=9.81
indata.g=g
indata.kxx=row.KXX
indata.kyy=row.KYY
indata.lcb=row.LCG
indata.lpp=row.lpp
indata.projectName=str(row.loading_condition_id)
rho=1000
indata.rho=rho
indata.zcg=row.kg-draught
#indata.waveFrequenciesMin=0.2
#indata.waveFrequenciesMax=0.5
#indata.waveFrequenciesIncrement=0.006
w=row.omega0/np.sqrt(row.scale_factor)
indata.waveFrequenciesMin=w*0.5
indata.waveFrequenciesMax=w*2.0
N=40
indata.waveFrequenciesIncrement=(indata.waveFrequenciesMax-indata.waveFrequenciesMin)/N
indata.runOptions["IE"].set_value(1)

In [ ]:
indata.runOptions["IG"].set_value(0)

In [ ]:
indata.speedMax

In [ ]:
indata.save('test.in')

In [ ]:
calculation = Calculation(outDataDirectory='scores2/result')
try:
    calculation.run(indata=indata)
except TDPError:
    print('Dissregarding the TDPError')

In [ ]:
output_file = OutputFile(filePath=calculation.outDataPath)
#df = output_file.get_result()
#
#fig,ax=plt.subplots()
#for index, group in df.groupby(by=['speed','wave direction'], sort=False):
#    group.plot(x='frequencies', y='rollAmplitude', style='o-', label=index, ax=ax)
#    
#ax.grid(True)
#ax.legend();
#ax.set_ylabel('Roll');

In [ ]:
w,B_W0=output_file.calculate_B_W0()

In [ ]:
fi_a = np.deg2rad(10)
w = row.omega0
scale_factor=row.scale_factor
V = row.ship_speed*1.852/3.6/np.sqrt(scale_factor)
R = 0.01*row.beam/scale_factor
lBK=row.BKL/scale_factor
bBK=row.BKB/scale_factor
ikeda = Ikeda.load_scoresII(V=V, w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                            scale_factor=scale_factor, lBK=lBK, bBK=bBK)

ikeda.R = R
ikeda.calculate_B44()

In [ ]:
ikeda_r = IkedaR.load_scoresII(V=V, w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                            scale_factor=scale_factor, lBK=lBK, bBK=bBK)

ikeda_r.calculate_B44()

In [ ]:
B_e = lambdas.B_e_lambda(B_1=row['B_1'], B_2=row['B_2'], phi_a=fi_a, 
                   omega0=row['omega0'])

volume=row.Volume/(scale_factor**3)
beam=row.beam/scale_factor

B_e_hat = lambdas.B_e_hat_lambda(B_e=B_e, Disp=volume, beam=beam, 
                                 g=g, rho=rho)
B_e_hat

In [ ]:
def calculate(inputs, ikeda):

    output = inputs.copy()
    output['B_44_hat'] = ikeda.calculate_B44()
    output['B_W0'] =ikeda.calculate_B_W0()
    output['B_W'] =ikeda.calculate_B_W()
    output['B_F'] =ikeda.calculate_B_F()
    output['B_E'] =ikeda.calculate_B_E()
    output['B_BK'] =ikeda.calculate_B_BK()
    output['B_L'] =ikeda.calculate_B_L()
    output['Bw_div_Bw0'] =ikeda.calculate_Bw_div_Bw0()
    
    return output

In [ ]:
scale_factor=row.scale_factor
inputs = pd.DataFrame()
inputs['Fn']=np.linspace(0,0.2,100)
inputs['lpp']=indata.lpp/scale_factor
inputs['V']=inputs['Fn']*np.sqrt(inputs['lpp']*g)

fi_a = np.deg2rad(10)
w = row.omega0

R = 0.01*row.beam/scale_factor
lBK=row.BKL/scale_factor
bBK=row.BKB/scale_factor
ikeda = Ikeda.load_scoresII(V=inputs['V'], w=w, fi_a=fi_a, indata=indata, output_file=output_file, 
                            scale_factor=scale_factor, lBK=lBK, bBK=bBK)
ikeda.R=R

In [ ]:
output = calculate(inputs=inputs, ikeda=ikeda)

fig,ax=plt.subplots()
output.plot.area(x='Fn', y = ['B_BK','B_F','B_E','B_L','B_W',], ax=ax)
ax.legend()
#ax.set_ylim(0,0.014)
#ax.set_title('Original Ikeda compared to model tests');